In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import transformers
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import csv
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import tqdm.notebook as tq
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
transformers.logging.set_verbosity(transformers.logging.CRITICAL)

ImportError: cannot import name 'SAVE_STATE_WARNING' from 'torch.optim.lr_scheduler' (C:\Users\stwan\Miniconda3\envs\GPT_Dialect\lib\site-packages\torch\optim\lr_scheduler.py)

In [78]:
df = pd.read_csv("runs/02 EMNLP SAE-AAVE Pairs/aave_gen_txt.csv")
df.head()

,index,txt
0,0,[' to be his sister who']
1,1,[' earth!!\n\nthe']
2,2,"["" 4:15 I'm""]"
3,3,[' TO GET HIM TO DO']
4,4,['\nnot giving up all']


In [49]:
df.shape

(2019, 2)

In [52]:
df.info

<bound method DataFrame.info of                                               first_seg  \
0     Sooo Manti Te'o was having a online/phone rela...   
1     this lil girl aint going to win im the king of...   
2     He up stairs rights now and I'm down here gett...   
3     Shit I Am Who Am..Fresh up out of Apologize..I...   
4     It's very rare that I get what I want. Now tha...   
...                                                 ...   
2014  At least I'm happy with knowing I'm always coo...   
2015  I love when you ad lip. If that's what you cal...   
2016  ISO a nice place to do karaoke or DJ music tha...   
2017  Must be an establishment with a "Grown Folks" ...   
2018  , so why i went to the store to get somethingg...   

                                         second_segs  
0                                 to be a female -_-  
1                          she better fall back ....  
2                               to go down nite nite  
3                            OR MOVE TH

## Tokenizer and Model

In [8]:
# PRE_TRAINED_MODEL_NAME = "nlptown/bert-base-multilingual-uncased-sentiment"
# tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
# model = AutoModelForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME)
# classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device=0)
classifier = pipeline('sentiment-analysis', device=0)

## Run BERT

In [89]:
df = pd.read_csv("runs/03 EMNLP SAE-AAVE Pairs/sae_gen_txt.csv")
with open('runs/03 EMNLP SAE-AAVE Pairs/sae_gen_txt_sentimental_analysis.csv', 'a', encoding="utf-8") as f:
    writer = csv.writer(f)
    for i, txt in enumerate(tq.tqdm(df.txt)):
        results = classifier(txt)
        for result in results:
            writer.writerow([i, result['label'], round(result['score'], 4)])

  0%|          | 0/2019 [00:00<?, ?it/s]

## Run VADER

In [102]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):
 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
 
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        return "Positive"
 
    elif sentiment_dict['compound'] <= - 0.05 :
        return "Negative"
 
    else :
        return "Neutral"

In [104]:
df = pd.read_csv("runs/03 EMNLP SAE-AAVE Pairs/sae_samples.tsv", sep="\t")
with open('runs/03 EMNLP SAE-AAVE Pairs/VADER/sae_second_seg_sentimental_analysis.csv', 'a', encoding="utf-8") as f:
    writer = csv.writer(f)
    for i, txt in enumerate(tq.tqdm(df.second_seg)):
        results = sentiment_scores(txt)
        writer.writerow([i, results])

  0%|          | 0/2019 [00:00<?, ?it/s]

## Run Text Blob

In [7]:
from textblob import TextBlob

def textblob_score(sentence):
    result = TextBlob(sentence)
    if result.sentiment.polarity == 0:
        return "Neutral"
    elif result.sentiment.polarity > 0:
        return "Positive"
    else:
        return "Negative"

In [14]:
df = pd.read_csv("runs/03 EMNLP SAE-AAVE Pairs/sae_gen_txt.csv")
with open('runs/03 EMNLP SAE-AAVE Pairs/TextBlob/sae_gen_txt_sentimental_analysis.csv', 'a', encoding="utf-8") as f:
    writer = csv.writer(f)
    for i, txt in enumerate(tq.tqdm(df.txt)):
        results = textblob_score(txt)
        writer.writerow([i, results])

  0%|          | 0/2019 [00:00<?, ?it/s]

## Stats

In [16]:
aave_first_pd = pd.read_csv("runs/03 EMNLP SAE-AAVE Pairs/TextBlob/aave_first_seg_sentimental_analysis.csv", header=None)
aave_second_pd = pd.read_csv("runs/03 EMNLP SAE-AAVE Pairs/TextBlob/aave_second_seg_sentimental_analysis.csv", header=None)
aave_gen_pd = pd.read_csv("runs/03 EMNLP SAE-AAVE Pairs/TextBlob/aave_gen_txt_sentimental_analysis.csv", header=None)
sae_first_pd = pd.read_csv("runs/03 EMNLP SAE-AAVE Pairs/TextBlob/sae_first_seg_sentimental_analysis.csv", header=None)
sae_second_pd = pd.read_csv("runs/03 EMNLP SAE-AAVE Pairs/TextBlob/sae_second_seg_sentimental_analysis.csv", header=None)
sae_gen_pd = pd.read_csv("runs/03 EMNLP SAE-AAVE Pairs/TextBlob/sae_gen_txt_sentimental_analysis.csv", header=None)

In [17]:
aave_first_pd.iloc[:, 1].value_counts()

Neutral     761
Positive    720
Negative    538
Name: 1, dtype: int64

In [18]:
aave_second_pd.iloc[:, 1].value_counts()

Neutral     1213
Positive     481
Negative     325
Name: 1, dtype: int64

In [19]:
aave_gen_pd.iloc[:, 1].value_counts()

Neutral     1583
Positive     290
Negative     146
Name: 1, dtype: int64

In [20]:
sae_first_pd.iloc[:, 1].value_counts()

Positive    782
Neutral     704
Negative    533
Name: 1, dtype: int64

In [21]:
sae_second_pd.iloc[:, 1].value_counts()

Neutral     1221
Positive     430
Negative     368
Name: 1, dtype: int64

In [22]:
sae_gen_pd.iloc[:, 1].value_counts()

Neutral     1551
Positive     305
Negative     163
Name: 1, dtype: int64